In [1]:
from dask import dataframe as dd

In [2]:
cols = ['Covered_Recipient_Type','Teaching_Hospital_CCN','Teaching_Hospital_ID','Teaching_Hospital_Name',
       'Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name','Physician_Last_Name','Physician_Name_Suffix',
       'Recipient_State','Physician_Primary_Type','Physician_Specialty','Physician_License_State_code1','Physician_License_State_code2',
       'Physician_License_State_code3','Physician_License_State_code4','Physician_License_State_code5',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Total_Amount_of_Payment_USDollars','Date_of_Payment','Number_of_Payments_Included_in_Total_Amount',
       'Form_of_Payment_or_Transfer_of_Value','Nature_of_Payment_or_Transfer_of_Value','City_of_Travel',
       'State_of_Travel', 'Country_of_Travel','Physician_Ownership_Indicator','Third_Party_Payment_Recipient_Indicator',
       'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value','Charity_Indicator','Third_Party_Equals_Covered_Recipient_Indicator',
       'Record_ID','Related_Product_Indicator','Covered_or_Noncovered_Indicator_1',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1',
       'Product_Category_or_Therapeutic_Area_1','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
       'Associated_Drug_or_Biological_NDC_1','Covered_or_Noncovered_Indicator_2',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Product_Category_or_Therapeutic_Area_2',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Associated_Drug_or_Biological_NDC_2',
       'Covered_or_Noncovered_Indicator_3',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Product_Category_or_Therapeutic_Area_3',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Associated_Drug_or_Biological_NDC_3',
       'Covered_or_Noncovered_Indicator_4',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Product_Category_or_Therapeutic_Area_4',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Associated_Drug_or_Biological_NDC_4',
       'Covered_or_Noncovered_Indicator_5',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5',
       'Product_Category_or_Therapeutic_Area_5',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
       'Associated_Drug_or_Biological_NDC_5','Recipient_Zip_Code']

In [3]:
#Load the data
sunshine = dd.read_csv('../Data/OP_DTL_GNRL_PGYR2019_P06302021.csv',dtype=str,usecols=cols)

In [4]:
# Dealing with missing data for recipient state
def licensedCA(row):
    if row['Recipient_State']=='CA' or row['Physician_License_State_code1']=='CA' or row['Physician_License_State_code1']=='CA' or row['Physician_License_State_code2']=='CA' or row['Physician_License_State_code3']=='CA' or row['Physician_License_State_code4']=='CA' or row['Physician_License_State_code5']=='CA':
        return True
    return False

sunshine['Licensed_In_CA'] = sunshine.apply(licensedCA,axis=1,meta=str)
        

In [5]:
#Filter only for california
cadf = sunshine[sunshine['Licensed_In_CA']==True]
del sunshine

In [7]:
%%time
# Hospital ID and Payment ID
hosp = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Teaching Hospital'][['Record_ID','Teaching_Hospital_ID']]
hosp.to_csv('../Data/Outputs_Cleanup/hospitals_payment.csv',single_file=True,index=False)

['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/hospitals_payment.csv']

In [8]:
%%time
# Hospitals and states
hosp = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Teaching Hospital'][['Recipient_State','Teaching_Hospital_ID']]
hosp = hosp.drop_duplicates(subset=['Teaching_Hospital_ID'])
hosp.to_csv('../Data/Outputs_Cleanup/hosp_states.csv',single_file=True,index=False)

['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/hosp_states.csv']

In [9]:
%%time
# Hospitals' other information
hosp = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Teaching Hospital'][['Teaching_Hospital_CCN','Teaching_Hospital_ID','Teaching_Hospital_Name']]
hosp = hosp.drop_duplicates(subset=['Teaching_Hospital_ID'])
hosp.to_csv('../Data/Outputs_Cleanup/hosp_info.csv',single_file=True,index=False)
del hosp

In [10]:
%%time
# Physician ID and Payment ID
physicians = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Physician'][['Record_ID','Physician_Profile_ID']]
physicians.to_csv('../Data/Outputs_Cleanup/physicians_payments.csv',single_file=True,index=False)

['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/physicians_payments.csv']

In [12]:
%%time
# Physicians and states
physicians = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Physician'][['Recipient_State','Physician_Profile_ID']]
physicians = physicians.drop_duplicates(subset=['Physician_Profile_ID'])
physicians.to_csv('../Data/Outputs_Cleanup/physicians_state.csv',single_file=True,index=False)

Wall time: 5min 50s


['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/physicians_state.csv']

In [13]:
%%time
# Physicians other information
physicians = cadf[cadf['Covered_Recipient_Type']=='Covered Recipient Physician'][['Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name','Physician_Last_Name','Physician_Name_Suffix',
       'Recipient_State','Physician_Primary_Type','Physician_Specialty','Physician_License_State_code1','Physician_License_State_code2',
       'Physician_License_State_code3','Physician_License_State_code4','Physician_License_State_code5','Recipient_Zip_Code']]
physicians = physicians.drop_duplicates(subset=['Physician_Profile_ID'])
physicians.to_csv('../Data/Outputs_Cleanup/physicians_info.csv',single_file=True,index=False)
del physicians

Wall time: 5min 49s


In [14]:
%%time
# Payment information
payments = cadf[['Record_ID','Total_Amount_of_Payment_USDollars','Date_of_Payment','Number_of_Payments_Included_in_Total_Amount',
                 'Form_of_Payment_or_Transfer_of_Value','Nature_of_Payment_or_Transfer_of_Value']]
payments.to_csv('../Data/Outputs_Cleanup/payments.csv',single_file=True,index=False)
del payments

Wall time: 5min 50s


In [15]:
%%time
# Pharmco and Record ID
pharmCo = cadf[['Record_ID','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID']]
pharmCo.to_csv('../Data/Outputs_Cleanup/pharmCo_payments.csv',single_file=True,index=False)

Wall time: 5min 52s


['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/pharmCo_payments.csv']

In [16]:
%%time
# Pharmco and state
pharmCo = cadf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State']]
pharmCo.to_csv('../Data/Outputs_Cleanup/pharmCo_states.csv',single_file=True,index=False)

Wall time: 5min 52s


['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/pharmCo_states.csv']

In [6]:
%%time
# Pharmco other information
pharmCo = cadf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID','Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
                'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name','Related_Product_Indicator']]
pharmCo.to_csv('../Data/Outputs_Cleanup/pharmCo_info.csv',single_file=True,index=False)

Wall time: 7min 12s


['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/pharmCo_info.csv']

In [7]:
%%time
# Pharmco other information
pharmCo = cadf[['Record_ID','Covered_or_Noncovered_Indicator_1',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1','Product_Category_or_Therapeutic_Area_1',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1','Associated_Drug_or_Biological_NDC_1','Covered_or_Noncovered_Indicator_2',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2','Product_Category_or_Therapeutic_Area_2',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2','Associated_Drug_or_Biological_NDC_2',
                'Covered_or_Noncovered_Indicator_3','Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3',
                'Product_Category_or_Therapeutic_Area_3','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
                'Associated_Drug_or_Biological_NDC_3','Covered_or_Noncovered_Indicator_4',
                'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4','Product_Category_or_Therapeutic_Area_4',
                'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4','Associated_Drug_or_Biological_NDC_4',
                'Covered_or_Noncovered_Indicator_5','Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5',
                'Product_Category_or_Therapeutic_Area_5','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5',
                'Associated_Drug_or_Biological_NDC_5']]
pharmCo.to_csv('../Data/Outputs_Cleanup/record_drugs.csv',single_file=True,index=False)

Wall time: 6min 13s


['D:/CLASSES/DSE203/Final Project/DSE-203-Knowledge-Graph/Data/Outputs_Cleanup/record_drugs.csv']